# Binary Classifier (BYOC) Deployment

- Classificador-binario
    - predictor.py: (Flask app for inference)
    - wsgi.py: (Wrapper around predictor)
    - nginx.conf: (Config for nginx front-end)
    - serve: program for container hosting, launches gunicorn server
    - Note that there is no train for pre-trained
- Dockerfile

# Push Docker Image to ECR

In [2]:
%%sh

# Name of algo -> ECR
algorithm_name=aidm-grupo-5-loan-default

cd container

#make serve executable
chmod +x Classificador-binario/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Região
region=$(aws configure get region)
region=${region:-eu-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"  #colocar semantic version

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/sagemaker-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credential-stores



Login Succeeded


DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            BuildKit is currently disabled; enable it by removing the DOCKER_BUILDKIT=0
            environment-variable.



Sending build context to Docker daemon  19.97kB


Error response from daemon: {"message":"Forbidden. Reason: [ImageBuild] 'sagemaker' is the only user allowed network input"}
Error response from daemon: No such image: aidm-grupo-5-loan-default:latest


# SageMaker Client Setup

- [SageMaker Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model): Model, Endpoint Config, and Endpoint Creation
- [SageMaker RunTime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint): Endpoint Invocation/Testing

In [21]:
import boto3
from sagemaker import get_execution_role
import pandas as pd
import json
from time import gmtime, strftime

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

#not really used in this use case, use when need to store model artifacts (Ex: MME)
s3_bucket = 'i32419'

role = get_execution_role()

# Model Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model)

In [22]:
model_name = 'loan-default-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = f"s3://{s3_bucket}/ai-deployment-monitoring-grupo-5/aidm-loan-default/model.tar.gz"
container = {
    "Image": container,
    "ModelDataUrl": model_url
}
algorithm_name = "aidm-grupo-5-loan-default"
#container = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, algorithm_name)
container = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{algorithm_name}:latest"
instance_type = 'ml.c5d.18xlarge'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: spacy-nermodel-2025-12-12-17-24-58
Model data Url: s3://i32419/spacy/
Container image: 267567228900.dkr.ecr.eu-west-1.amazonaws.com/andrecastroalgo:latest
Model Arn: arn:aws:sagemaker:eu-west-1:267567228900:model/spacy-nermodel-2025-12-12-17-24-58


# Endpoint Config Creation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)

In [23]:
endpoint_config_name = 'loan-default-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: spacy-ner-config2025-12-12-17-25-08
Endpoint config Arn: arn:aws:sagemaker:eu-west-1:267567228900:endpoint-config/spacy-ner-config2025-12-12-17-25-08


# Endpoint Creation
[Documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint)

In [24]:
%%time

import time

endpoint_name = 'loan-default-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: spacy-ner-endpoint2025-12-12-17-25-12


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/IPython/core/magics/execution.py:1370 in time            │
│                                                                                                  │
│   1367 │   │   else:                                                                             │
│   1368 │   │   │   st = clock2()                                                                 │
│   1369 │   │   │   try:                                                                          │
│ ❱ 1370 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1371 │   │   │   │   out = None                                                                │
│   1372 │   │   │   │   # multi-line %%time case                                                  │
│   1373 │   │   │   │   if expr_val is not None:                                                  │
│ in <module>:6                                                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:602 in _api_call                      │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/context.py:123 in wrapper                       │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1078 in _make_api_call                │
│                                                                                                  │
│   1075 │   │   │   │   'error_code_override'                                                     │
│   1076 │   │   │   ) or error_info.get("Code")                                                   │
│   1077 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1078 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1079 │   │   else:                                                                             │
│   1080 │   │   │   return parsed_response                  

# Endpoint Invocation

[Documentation](https://boto3.amazonaws.com/v1/documentation/api/1.9.42/reference/services/sagemaker-runtime.html#SageMakerRuntime.Client.invoke_endpoint)

In [ ]:
df = pd.read_csv("AIDM-Grupo-5/Dataset/Loan_Default.csv")
X = df.drop(columns=["Status"])

sample = X.iloc[:1].to_dict(orient="records")  # 1 linha
request_body = {"instances": sample}           # ou {"input": sample}

payload = json.dumps(request_body)

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=payload
)

result = json.loads(response["Body"].read().decode("utf-8"))
result

# Delete Endpoint

Before leaving this exercise, it is a good practice to delete the resources created.

In [ ]:
# sm_client.delete_endpoint(EndpointName=endpoint_name)
# sm_client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)
# sm_client.delete_model(ModelName=ModelName)